Experiment settings

In [1]:
import simpy
import numpy as np
import pandas as pd
import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_max = 5
order_time_min = 1
food_prepare_min = 2
food_prepare_max = 6

n_kitchen = 2
n_counter = 2
n_customer = 100
SIM_TIME = 12 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [2]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_service_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Service Start Time","Queue time","Exit Time","Total Wait Time","Total Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) FIFO Algorithim

In [3]:
from functools import partial, wraps
def patch_resource(resource, pre=None, post=None):
     """Patch *resource* so that it calls the callable *pre* before each
     put/get/request/release operation and the callable *post* after each
     operation.  The only argument to these functions is the resource
     instance.

     """
     def get_wrapper(func):
         # Generate a wrapper for put/get/request/release
         @wraps(func)
         def wrapper(*args, **kwargs):
             # This is the actual wrapper
             # Call "pre" callback
             if pre:
                 pre(resource)

             # Perform actual operation
             ret = func(*args, **kwargs)

             # Call "post" callback
             if post:
                 post(resource)

             return ret
         return wrapper

     # Replace the original operations with our wrapper
     for name in ['take_order', 'receive_order', 'request', 'release']:
         if hasattr(resource, name):
             setattr(resource, name, get_wrapper(getattr(resource, name)))
             
def monitor(data, resource):
     """This is our monitoring callback."""
     item = (
         resource.env.now,  # The current simulation time
         resource.count,  # The number of users
         len(resource.queue),  # The number of queued processes
     )
     data.append(item)

In [4]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        self.service_start = None
        
    def take_order(self,cus,env):
        self.service_start = env.now
        print("%s is placing order at counter %.2f" %(cus,self.service_start))
        time_taken_to_place_order = np.random.uniform(order_time_min, order_time_max)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # yield env.process(tank_truck(env, fuel_pump))

    def receive_order(self,cus,env,resource):
        global counter_total_service_times

        with resource.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(resource.prepare_food(cus,env))
            service_end = env.now
            print("%s collected the food at %.2f" %(cus, service_end))
            # Record idle counter and add to total count
            counter_total_service_times += (service_end-self.service_start)

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = np.random.uniform(food_prepare_min,food_prepare_max)
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,1] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,2]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env))
        # waiting order at counter
        prepare_food_start = env.now
        # counter is idle now
        yield env.process(queue.receive_order(label,env,kitchen))
        # prepare_food_end = round_down(env.now)
        # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
        # receive food from counter
        prepare_food_end = env.now
        data[label,3] = round_down(prepare_food_end)

        kitchen_prepare_duration = (prepare_food_end-prepare_food_start)
        # total wait time
        data[label,4] = round_down(kitchen_prepare_duration+queue_time)
        # total service time
        data[label,5] = round_down(prepare_food_end-service_start)
        # total time in system
        data[label,6] = round_down(prepare_food_end-arrive_time)


## Start FIFO Simulation

In [5]:
counter = Counter(env,n_counter)
kitchen = Kitchen(env,n_kitchen)
# Simlating possion process for customer arrival
def customer_arrivals(env,res_counter):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    for i in range(n_customer):
        yield env.timeout(random.expovariate(1 / 5))
        env.process(customer(env, i+1, res_counter, kitchen, result_fifo))

env.process(customer_arrivals(env,counter))
env.run(until=SIM_TIME)

0 entering the queue at 5.10
0 is placing order at counter 5.10
1 entering the queue at 5.23
1 is placing order at counter 5.23
2 entering the queue at 6.84
Order of 0 sent to kitchen at 7.29
Kitchen is preparing food for 0 at 7.29
3 entering the queue at 8.10
Order of 1 sent to kitchen at 9.03
Kitchen is preparing food for 1 at 9.03
Cooked food for 0 at 10.91
0 collected the food at 10.91
2 is placing order at counter 10.91
Cooked food for 1 at 11.98
1 collected the food at 11.98
3 is placing order at counter 11.98
4 entering the queue at 14.77
Order of 2 sent to kitchen at 15.01
Kitchen is preparing food for 2 at 15.01
Order of 3 sent to kitchen at 16.86
Kitchen is preparing food for 3 at 16.86
Cooked food for 2 at 17.79
2 collected the food at 17.79
4 is placing order at counter 17.79
Order of 4 sent to kitchen at 19.98
Kitchen is preparing food for 4 at 19.98
5 entering the queue at 20.41
Cooked food for 3 at 20.45
3 collected the food at 20.45
5 is placing order at counter 20.45
O

## FIFO Result & Analysis

In [6]:

# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)

total_wait_time = np.sum(df_fifo["Total Wait Time"])
total_time_in_system = np.sum(df_fifo["Total Time in System"])
total_service_time = np.sum(df_fifo['Total Service Time'])
print("")

def display(i):
    print()
    print("Total Simulation Time=> %.2f Minutes" % SIM_TIME)
    print("Counter Total Idle Time => %.2f Minutes" % (SIM_TIME - counter_total_service_times))
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_customer)



Total Simulation Time=> 720.00 Minutes
Counter Total Idle Time => 330.94 Minutes

Average Waiting Time => 8.50 Minutes
Average Service Time => 6.94 Minutes
Average Time Spent In System => 11.44 Minutes
